# IPL Score **Prediction**

In [1]:
# Step 1: Mount Google Drive (Colab only)
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install necessary libraries
!pip install pandas scikit-learn gradio



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/processed_ipl_data2.csv', low_memory=False)

# Clean and filter
df = df[df['inning'] == 1]  # Only 1st innings
df.dropna(subset=['batting_team', 'bowling_team', 'total_runs'], inplace=True)

# Calculate current score, wickets, and over
df['current_score'] = df.groupby('match_id')['total_runs'].cumsum()
df['wickets'] = df.groupby('match_id')['is_wicket'].cumsum()
df['current_over'] = df['over'] + df['ball'] / 6

# Get final score of each match
final_scores = df.groupby('match_id')['current_score'].max().reset_index()
final_scores.columns = ['match_id', 'final_score']

# Merge final scores with each ball-level row
data = df.merge(final_scores, on='match_id')

# Add additional features
data['run_rate'] = data['current_score'] / (data['current_over'] + 0.1)
data['balls_left'] = 120 - (data['current_over'] * 6)
data['wickets_left'] = 10 - data['wickets']

# Final features for ML
data = data[['batting_team', 'bowling_team', 'current_over', 'current_score', 'wickets', 'run_rate', 'balls_left', 'wickets_left', 'final_score']]


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time

# Sample only 10% of data for speed
sampled_data = data.sample(frac=0.1, random_state=42)

X = sampled_data.drop('final_score', axis=1)
y = sampled_data['final_score']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encoding
categorical_features = ['batting_team', 'bowling_team']
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
], remainder='passthrough')

# Simple Random Forest for fast training
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
        n_estimators=10,      # Much faster
        max_depth=5,          # Shallow depth
        n_jobs=-1,
        random_state=42
    ))
])

# Time it
start_time = time.time()
pipeline.fit(X_train, y_train)
end_time = time.time()

print(f"✅ Model trained in {end_time - start_time:.2f} seconds")


✅ Model trained in 0.19 seconds


In [4]:
import gradio as gr

# Prediction function
def predict_score(batting_team, bowling_team, current_score, current_over, wickets):
    run_rate = current_score / (current_over + 0.1)
    balls_left = 120 - (current_over * 6)
    wickets_left = 10 - wickets

    input_data = pd.DataFrame({
        'batting_team': [batting_team],
        'bowling_team': [bowling_team],
        'current_over': [current_over],
        'current_score': [current_score],
        'wickets': [wickets],
        'run_rate': [run_rate],
        'balls_left': [balls_left],
        'wickets_left': [wickets_left]
    })

    predicted_score = pipeline.predict(input_data)[0]
    return f"🏏 Predicted Final Score: {int(predicted_score)} runs"

# Teams list
teams = sorted(data['batting_team'].dropna().unique())

# Gradio interface
interface = gr.Interface(
    fn=predict_score,
    inputs=[
        gr.Dropdown(choices=teams, label="Batting Team"),
        gr.Dropdown(choices=teams, label="Bowling Team"),
        gr.Number(label="Current Score", value=50),
        gr.Number(label="Overs Completed", value=7),
        gr.Number(label="Wickets Fallen", value=2)
    ],
    outputs="text",
    title="IPL Final Score Predictor",
    description="Enter live match stats to predict the 1st innings final score!",
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e902d1470e783b795.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
